# Distribution Grid Optimization with Genetic Algorithm

This notebook optimizes the placement of distributed generators and loads in a distribution network to minimize line loading.

In [ ]:
import pandapower as pp
import pandapower.plotting as plot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from grid_optimizer import GridOptimizer

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

%matplotlib inline

## 1. Create Distribution Network

In [ ]:
def create_distribution_network():
    """
    Create 18-bus distribution network with transformer.
    
    Topology:
    - Bus 1: HV bus (20 kV) with external grid
    - Buses 2-18: LV buses (0.4 kV)
    - Two feeders from Bus 2 (transformer secondary)
    """
    net = pp.create_empty_network()
    
    # Create HV bus
    pp.create_bus(net, vn_kv=20.0, name="HV Bus", index=1)
    
    # Create LV buses with loads and generators
    for i in range(2, 19):
        pp.create_bus(net, vn_kv=0.4, name=f"Bus {i}", index=i)
        pp.create_load(net, bus=i, p_mw=0.01, q_mvar=0.005, name=f"Load {i-2}")
        pp.create_sgen(net, bus=i, p_mw=0.01, q_mvar=0.005, name=f"Gen {i-2}")
    
    # External grid connection
    pp.create_ext_grid(net, bus=1, vm_pu=1.02, name="Grid Connection")
    
    # Transformer
    pp.create_transformer(
        net, hv_bus=1, lv_bus=2, 
        std_type="0.4 MVA 20/0.4 kV", 
        name="Transformer"
    )
    
    # Feeder 1: Buses 2->3->4->...->9
    pp.create_line(
        net, from_bus=2, to_bus=3, length_km=0.100, 
        std_type="NAYY 4x50 SE", name="Line 2-3"
    )
    for i in range(3, 9):
        pp.create_line(
            net, from_bus=i, to_bus=i+1, length_km=0.100,
            std_type="NAYY 4x50 SE", name=f"Line {i}-{i+1}"
        )
    
    # Feeder 2: Buses 2->10->11->...->18
    pp.create_line(
        net, from_bus=2, to_bus=10, length_km=0.100,
        std_type="NAYY 4x50 SE", name="Line 2-10"
    )
    for i in range(10, 18):
        pp.create_line(
            net, from_bus=i, to_bus=i+1, length_km=0.100,
            std_type="NAYY 4x50 SE", name=f"Line {i}-{i+1}"
        )
    
    return net

# Create network
net = create_distribution_network()
print(f"Network created with {len(net.bus)} buses, {len(net.line)} lines")
print(f"Number of loads: {len(net.load)}")
print(f"Number of generators: {len(net.sgen)}")

## 2. Visualize Network Topology

In [ ]:
# Plot network
plot.simple_plot(net, plot_loads=True, plot_sgens=True)
plt.title('Distribution Network Topology')
plt.show()

# Run initial power flow
pp.runpp(net)
print("\nInitial power flow results:")
print(net.res_ext_grid)

## 3. Load Generation and Load Data

In [ ]:
# Load time series data
gen_data = pd.read_csv("GenerationData_B.csv", index_col=0)
gen_data.columns.name = "household"

load_data = pd.read_csv("LoadData_B.csv", index_col=0)
load_data.columns.name = "household"

print(f"Generation data shape: {gen_data.shape}")
print(f"Load data shape: {load_data.shape}")
print(f"\nTime steps: {len(gen_data)} hours")
print(f"Households: {len(gen_data.columns)}")

# Visualize daily profiles
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(gen_data.mean(axis=1), linewidth=2, color='green')
ax1.set_xlabel('Hour of Day')
ax1.set_ylabel('Average Generation (MW)')
ax1.set_title('Average Generation Profile')
ax1.grid(True, alpha=0.3)

ax2.plot(load_data.mean(axis=1), linewidth=2, color='red')
ax2.set_xlabel('Hour of Day')
ax2.set_ylabel('Average Load (MW)')
ax2.set_title('Average Load Profile')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Run Genetic Algorithm Optimization

In [ ]:
# Create optimizer
optimizer = GridOptimizer(
    gen_data=gen_data,
    load_data=load_data,
    net=net,
    population_size=20,
    mutation_rate=0.15,
    elite_size=2,
    max_generations=50,
    convergence_threshold=0.001,
    verbose=True
)

# Run optimization
best_gen_order, best_load_order, best_fitness = optimizer.optimize()

print("\n" + "="*60)
print("OPTIMIZATION RESULTS")
print("="*60)
print(f"Best Fitness (Total Line Loading): {best_fitness:.4f}")
print(f"\nOptimal Generator Allocation: {best_gen_order}")
print(f"Optimal Load Allocation: {best_load_order}")

## 5. Visualize Optimization Progress

In [ ]:
# Plot convergence
optimizer.plot_fitness_history()

## 6. Compare Random vs Optimized Allocation

In [ ]:
from utils import run_time_series

# Random allocation
random_gen = list(range(17))
random_load = list(range(17))
import random as rnd
rnd.shuffle(random_gen)
rnd.shuffle(random_load)

print("Evaluating random allocation...")
res_ext_random, res_lines_random = run_time_series(
    gen_data, load_data, net,
    index_order_gen=pd.Index(random_gen),
    index_order_load=pd.Index(random_load)
)

print("Evaluating optimized allocation...")
res_ext_opt, res_lines_opt = run_time_series(
    gen_data, load_data, net,
    index_order_gen=pd.Index(best_gen_order),
    index_order_load=pd.Index(best_load_order)
)

# Compare results
random_fitness = res_lines_random.max(axis=1).sum()
opt_fitness = res_lines_opt.max(axis=1).sum()
improvement = ((random_fitness - opt_fitness) / random_fitness) * 100

print("\n" + "="*60)
print("COMPARISON")
print("="*60)
print(f"Random allocation fitness: {random_fitness:.4f}")
print(f"Optimized allocation fitness: {opt_fitness:.4f}")
print(f"Improvement: {improvement:.2f}%")

## 7. Visualize Line Loading Comparison

In [ ]:
# Plot line loading over time
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Random allocation
for col in res_lines_random.columns:
    axes[0].plot(res_lines_random.index, res_lines_random[col], alpha=0.6)
axes[0].set_ylabel('Line Loading (%)')
axes[0].set_title('Random Allocation - Line Loading Over Time')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=100, color='r', linestyle='--', label='Thermal Limit')
axes[0].legend()

# Optimized allocation
for col in res_lines_opt.columns:
    axes[1].plot(res_lines_opt.index, res_lines_opt[col], alpha=0.6)
axes[1].set_xlabel('Hour of Day')
axes[1].set_ylabel('Line Loading (%)')
axes[1].set_title('Optimized Allocation - Line Loading Over Time')
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=100, color='r', linestyle='--', label='Thermal Limit')
axes[1].legend()

plt.tight_layout()
plt.savefig('line_loading_comparison.png', dpi=150)
plt.show()

## 8. Analyze External Grid Power

In [ ]:
# Compare external grid power
plt.figure(figsize=(12, 5))
plt.plot(res_ext_random.index, res_ext_random.iloc[:, 0], 
         label='Random', linewidth=2, marker='o', markersize=4)
plt.plot(res_ext_opt.index, res_ext_opt.iloc[:, 0], 
         label='Optimized', linewidth=2, marker='s', markersize=4)
plt.xlabel('Hour of Day')
plt.ylabel('External Grid Power (MW)')
plt.title('External Grid Power Draw Comparison')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('external_grid_comparison.png', dpi=150)
plt.show()

print(f"Random - Peak grid import: {res_ext_random.max().max():.4f} MW")
print(f"Optimized - Peak grid import: {res_ext_opt.max().max():.4f} MW")

## 9. Save Results

In [ ]:
# Save optimal allocation
results = {
    'optimal_gen_order': best_gen_order,
    'optimal_load_order': best_load_order,
    'best_fitness': best_fitness,
    'improvement_vs_random': improvement
}

import json
with open('optimization_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("Results saved to 'optimization_results.json'")